In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from pandas import DataFrame
from pandas import concat
import itertools
import xgboost as xgb

In [2]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = [], []
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def mean_percent_error(y_pred, y_true):
    return np.sum(np.abs(y_pred - y_true) / y_true) / y_true.shape[0]

In [21]:
PATH = './all_data.csv'
aqi_data = pd.read_csv(PATH)
aqi_data = aqi_data.drop(['Unnamed: 0', 'date'], axis=1)
feature_columns = aqi_data.columns
target_columns = ['PM2_5', 'PM_10', 'SO2', 'NO2', 'O3', 'CO']
for pollutant in range(len(target_columns)):
    for lag in range(1, 6):
        aqi_data_supervised = series_to_supervised(aqi_data, 24, lag, True)
        X = aqi_data_supervised[aqi_data_supervised.columns[0: feature_columns.__len__() * 24]]
        aqi_data_y = series_to_supervised(aqi_data[target_columns], 24, lag, True)
        y = aqi_data_y[aqi_data_y.columns[-6+pollutant:-5+pollutant]]
        train_X, test_X, train_y, test_y = train_test_split(X, y, train_size=0.9, shuffle=False)
        scaler_X = MinMaxScaler()
        scaler_y = MinMaxScaler()
        train_X = scaler_X.fit_transform(train_X)
        test_X = scaler_X.transform(test_X)
        train_y = scaler_y.fit_transform(train_y)
        test_y = scaler_y.transform(test_y)
        estimators = [ i for i in range(50, 101, 50)]
        models1 = [('RandomForest %d' % estimator, RandomForestRegressor(n_estimators=estimator, n_jobs=5, random_state=1, max_depth=20)) for estimator in estimators]
        models2 = [('GradientBoosting %d' % estimator, GradientBoostingRegressor(n_estimators=estimator, random_state=1, max_depth=20)) for estimator in estimators]
        model3 = [('LinearRegression', LinearRegression(n_jobs=5))]
        models4 = [('XGBoost %d' % estimator, xgb.XGBRegressor(n_jobs=5, n_estimators=estimator, random_state=1, max_depth=-1)) for estimator in estimators]
        models = list(itertools.chain.from_iterable([model3]))
        for model in models:
            model[1].fit(train_X, np.ravel(train_y))
        metric = pd.DataFrame(columns=['mse', 'mae', 'r2', 'mape'])
        for model in models:
            pred = model[1].predict(test_X)
            pred = scaler_y.inverse_transform(pred.reshape(-1,1))
            test_y = scaler_y.inverse_transform(test_y)
            mse = ('mse', mean_squared_error(test_y, pred))
            mae = ('mae', mean_absolute_error(test_y, pred))
            mape = ('mape', mean_percent_error(pred, test_y))
            r2 = ('r2', r2_score(test_y, pred))
            for err in [mse , mae, mape, r2]:
                metric.loc[model[0], err[0]] = err[1]
            print(mse, mae, mape, r2, lag, target_columns[pollutant])
    # metric.to_csv('./a.csv', sep=',', header=True, index=True)


('mse', 74.21195286607337) ('mae', 4.986946475624938) ('mape', 0.10258095525233342) ('r2', 0.9664660699546058) 1 PM2_5
('mse', 172.26411074076174) ('mae', 8.339670181529591) ('mape', 0.17331276810172092) ('r2', 0.9221595387829553) 2 PM2_5
('mse', 300.5813839995832) ('mae', 11.444309446368814) ('mape', 0.24502663374944736) ('r2', 0.8641772017213983) 3 PM2_5
('mse', 436.2234278300893) ('mae', 14.165774554689367) ('mape', 0.31173290995840025) ('r2', 0.8028850427987632) 4 PM2_5
('mse', 568.8561247428657) ('mae', 16.51627746398364) ('mape', 0.3707910707486178) ('r2', 0.7429527083400331) 5 PM2_5
('mse', 207.67631853304044) ('mae', 8.079824321285187) ('mape', 0.08780101364430504) ('r2', 0.9188372213435057) 1 PM_10
('mse', 509.3785056218757) ('mae', 12.962473469240985) ('mape', 0.14238300458519945) ('r2', 0.8009278323296801) 2 PM_10
('mse', 828.4301020370532) ('mae', 16.976362388842055) ('mape', 0.18982837924302873) ('r2', 0.6762380541076802) 3 PM_10
('mse', 1100.6928414435442) ('mae', 20.0415

ValueError: at least one array or dtype is required

In [16]:
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    train_X = scaler_X.fit_transform(train_X)
    test_X = scaler_X.transform(test_X)
    train_y = scaler_y.fit_transform(train_y)
    test_y = scaler_y.transform(test_y)

In [17]:
    estimators = [ i for i in range(50, 101, 50)]
    models1 = [('RandomForest %d' % estimator, RandomForestRegressor(n_estimators=estimator, n_jobs=5, random_state=1, max_depth=20)) for estimator in estimators]
    models2 = [('GradientBoosting %d' % estimator, GradientBoostingRegressor(n_estimators=estimator, random_state=1, max_depth=20)) for estimator in estimators]
    model3 = [('LinearRegression', LinearRegression(n_jobs=5))]
    models4 = [('XGBoost %d' % estimator, xgb.XGBRegressor(n_jobs=5, n_estimators=estimator, random_state=1, max_depth=-1)) for estimator in estimators]
    models = list(itertools.chain.from_iterable([model3]))
    for model in models:
        model[1].fit(train_X, np.ravel(train_y))

In [14]:
    metric = pd.DataFrame(columns=['mse', 'mae', 'r2', 'mape'])
    for model in models:
        pred = model[1].predict(test_X)
        pred = scaler_y.inverse_transform(pred.reshape(-1,1))
        test_y = scaler_y.inverse_transform(test_y)
        mse = ('mse', mean_squared_error(test_y, pred))
        mae = ('mae', mean_absolute_error(test_y, pred))
        mape = ('mape', mean_percent_error(pred, test_y))
        r2 = ('r2', r2_score(test_y, pred))
        for err in [mse , mae, mape, r2]:
            metric.loc[model[0], err[0]] = err[1]
    metric.to_csv('./a.csv', sep=',', header=True, index=True)